<a href="https://colab.research.google.com/github/SitthisakMoukomla/earthengine-community/blob/master/%E0%B9%82%E0%B8%AB%E0%B8%A5%E0%B8%94_%E0%B8%82%E0%B9%89%E0%B8%AD%E0%B8%A1%E0%B8%B9%E0%B8%A5_Landsat_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **มาลอง Load ข้อมูล Landsat-9 **

**Authenticate and Initialize Earth Engine**

Earth Engine API ได้รับการติดตั้งโดยค่าเริ่มต้นใน Google Colaboratory ดังนั้นจึงต้องนำเข้าและ Authenticate ก่อน และต้องดำเนินการขั้นตอนเหล่านี้ให้เสร็จสิ้น

In [ ]:
!pip install geemap
import ee
import geemap

# Initialize the Earth Engine Python API
# Trigger the authentication flow.
ee.Authenticate()
# Initialize the library.
ee.Initialize()

In [35]:
# Create an interactive map
Map = geemap.Map(center=[14, 101], zoom=8)

# Load countries data
Thailand = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_na', 'Thailand'))

# Create an ImageCollection for Landsat data
collection = (ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
  .filterDate('2022-12-01', '2023-04-15')
  .filterBounds(Thailand)
  .sort('CLOUD_COVER'))

# Compute the median image from the collection
c = collection.median()

# Define a function to apply scaling factors to the image
def applyScaleFactors(image):
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_*.*').multiply(0.00341802).add(149.0)
    return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)

# Apply scaling factors to the median image
dataset = applyScaleFactors(c)

# Define visualization parameters
visualization = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.0,
    'max': 0.25,
}

# Add the dataset as a layer to the map
Map.addLayer(dataset.clip(Thailand), visualization, 'True Color (432)')

# Display the map
Map


Map(center=[14, 101], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chi…